node_a_client
============

In [1]:
#! /usr/bin/env python
import rospy
import jupyros as jr
import assignment_2_2022.msg
import actionlib 
import ipywidgets as widgets
from nav_msgs.msg import Odometry
from my_assignment_2.msg import Info
from sensor_msgs.msg import LaserScan
from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt
import numpy as np
global number_reached
global number_canceled

number_reached = 0
number_canceled = 0


In [2]:
# initialize the node 
rospy.init_node("node_a")

client = actionlib.SimpleActionClient("/reaching_goal", assignment_2_2022.msg.PlanningAction)

client.wait_for_server()

True

In [3]:
new_goal = assignment_2_2022.msg.PlanningGoal()

# create two boxes in order to input the coordinates of a new goal

x_goal = widgets.FloatText(description = "x value:")
y_goal = widgets.FloatText(description = "y value:")

# create a button to send the goal

send_button = widgets.Button(description = "SEND THE GOAL")

# create a button to stop the robot before it arrives to the defined goal

stop_button = widgets.Button(description = "STOP!", disabled = True)

# create a widget to display the status of the goals

goals_status = widgets.Textarea(value = "Goals status:\n ", disabled  = True)

# create a function in order to define a new goal 

def define_goal(g):
    
    # check if the input is valid
    
    new_goal.target_pose.pose.position.x = x_goal.value
    new_goal.target_pose.pose.position.y = y_goal.value
    
    # send the goal to the client 

    client.send_goal(new_goal)
    x_goal.disabled = True
    y_goal.disabled = True
    send_button.disabled = True
    stop_button.disabled = False

    goals_status.value = goals_status.value + "Goal with: x= " + str(x_goal.value) + " and y= " + str(y_goal.value)
    
# create a function in order to stop the robot cancelling the goal

def stop_robot(s):
    
    client.cancel_goal()
    
# call the function define_goal when the send_button is pressed

send_button.on_click(define_goal)

# call the function stop_robot when the stop_button is pressed

stop_button.on_click(stop_robot)
    
# show the widgets

widgets.HBox([widgets.VBox([x_goal,y_goal,goals_status]), widgets.VBox([send_button, stop_button])])
    

In [4]:
# create a function to return if a certain goal has been reached correctly or not

def goal_status_callback(goal_stat):
    
    if goal_stat.feedback.stat == "Target reached!":
        
        global number_reached
        number_reached += 1
        x_goal.disabled = False
        y_goal.disabled = False
        send_button.disabled = False
        stop_button.disabled = True
    
        goals_status.value = goals_status.value + " -> Reached\n"
        
    elif goal_stat.feedback.stat == "Target cancelled!":
        
        global number_canceled
        number_canceled += 1
        x_goal.disabled = False
        y_goal.disabled = False
        send_button.disabled = False
        stop_button.disabled = True
        
        goals_status.value = goals_status.value + " -> Cancelled\n"


In [5]:
jr.subscribe("/reaching_goal/feedback", assignment_2_2022.msg.PlanningActionFeedback, goal_status_callback)

node_a_publisher
===============

In [6]:
# publisher for the information about the robot's position and velocity
# the custom message Info contains the fields x,y (corresponding to the position) and the fields vel_x, vel_y (for the velocities)

publisher = rospy.Publisher("/InfoRobot", Info, queue_size = 10)

Info about Robot's position
=======================

In [7]:
# create two widgets for the robot position

pos_x = widgets.FloatText(description = "pos_x: ", disabled = True)
pos_y = widgets.FloatText(description = "pos_y: ", disabled = True)

def info_robot_position(info):
    
    pos_x.value = info.x
    pos_y.value = info.y
    
widgets.HBox([pos_x,pos_y])


In [8]:
jr.subscribe('/InfoRobot', Info, info_robot_position)

Closest Obstacle
==============

In [9]:
# create a widget to display the distance between the robot and the closest obstacle near it

scan_closest = widgets.FloatText(description = "distance from the closest obstacle", disabled = True)

# create a function to evaluate the distance between the robot and the closest obstacle

def closest(close):
    
    min = 30
    
    for x in close.ranges: 
        if x > 0.1 and x < min:
            min = x
    scan_closest.value = min 
    
display(scan_closest)


FloatText(value=0.0, description='distance from the closest obstacle', disabled=True)

In [10]:
jr.subscribe("/scan", LaserScan, closest)

PLOT THE POSITION
==================

In [11]:
class Visualiser:
    def __init__(self):
        self.fig, self.ax = plt.subplots() 
        self.ln, = plt.plot([], [], 'ro') 
        self.x_data, self.y_data = [] , []
    def plot_init(self): 
        self.ax.set_xlim(-10, 10)
        self.ax.set_ylim(-10, 10) 
        return self.ln
    def odom_callback(self, msg): 
        self.y_data.append(msg.pose.pose.position.y) 
        self.x_data.append(msg.pose.pose.position.x)
        
        # define the variable robot_info as the same type of the custom message
    
        robot_info = Info()

        # fill all the fields of robot_info with the informations of the robot

        robot_info.x = msg.pose.pose.position.x
        robot_info.y = msg.pose.pose.position.y

        robot_info.vel_x = msg.twist.twist.linear.x
        robot_info.vel_y = msg.twist.twist.linear.y

        publisher.publish(robot_info)
    def update_plot(self, frame): 
        self.ln.set_data(self.x_data,  self.y_data) 
        return self.ln

In [12]:
%matplotlib widget
vis = Visualiser()
sub = jr.subscribe('/odom',  Odometry, vis.odom_callback)
ani = FuncAnimation(vis.fig, vis.update_plot,  init_func=vis.plot_init) 
plt.show(block=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Keep track of the reached/canceled goals
=============================

In [13]:
update_button = widgets.Button(description = "Update the bar chart", button_style = "info")

plt.figure(2)
columns = [number_reached, number_canceled]
rows = np.linspace(1,2,2)
plt.xticks(np.linspace(0,12,13))
plt.xlabel('Goals Reached                                                                    Goals Canceled')
plt.bar(rows,columns, align='edge', width=0.10, color='green')
plt.show()

def update_button_click(ub):
    
    plt.figure(2)
    columns = [number_reached, number_canceled]
    rows = np.linspace(1,2,2)
    plt.xticks(np.linspace(0,12,13))
    plt.xlabel('Goals Reached                                                                    Goals Canceled')
    plt.bar(rows,columns, align='edge', width=0.10, color='green')
    plt.show()
    
update_button.on_click(update_button_click)
display(update_button)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Button(button_style='info', description='Update the bar chart', style=ButtonStyle())